In [3]:
import os
import openai
import datetime

# Set up the OpenAI API key
# Option 1: Set API key directly in the code (quick but not secure for production)
#openai.api_key = "your-api-key-here"

# Option 2: Use environment variable (more secure)
api_key = os.environ.get("DEEPSEEK_API_KEY")

def query_deepseek(prompt, model="deepseek-reasoner"):
    """
    Send a query to the DeepSeek API and return the response.
    
    Args:
        prompt (str): The text prompt to send to the API
        model (str): The model to use
        
    Returns:
        str: The model's response text
    """
    try:
        client = openai.OpenAI(api_key=api_key, base_url="https://api.deepseek.com")
        current_time = datetime.datetime.now().strftime("%Y-%m-%d %H:%M")
        schedule = """事项: 入职培训
日期: 2025-03-3
时间段: 8:00-9:00
类型: 固定日程
重要程度：4
"""
        prompt = f"""
【处理规则】

时间分配：
每日专注时段（默认 09:00-12:30;14:00-17:30）
单任务最长持续≤120分钟，间隔≥10分钟缓冲
冲突处理原则：截止时间近者优先，重要度相同则用时少者优先

【特殊规则】

自动预留截止时间前15%时长作为应急缓冲
连续工作>120分钟必须插入15分钟休息
每日保留19:00-21:00作为弹性时段（可调整）
输出时间精度保持±5分钟对齐

【输入示例】
当前时间：2024-03-14 16:00

当前时间表：
事项: 项目会议
日期: 2024-03-15
时间段: 09:00-10:30
类型: 固定日程
截止日期：2024-03-15
重要程度：5

事项: 完成报告
日期: 2024-03-16
时间段: 15:30-16:30
类型: 任务事项
截止日期：2024-03-18
重要程度：3

新增任务：
我后天下午三点要开个会，得准备个PPT


【输出示例】
新任务：开会并准备PPT
开会并准备PPT → 拆分为：
- 准备演讲稿（预计需两小时，时段：2024-03-15 15:00-17:00）
- 进行会议（预计需一小时，时段：2024-03-16 15:00-16:00）


日程建议：
事项: 完成报告
日期: 2024-03-17
时间段: 9:00-10:00
类型: 任务事项
截止日期：2024-03-18
重要程度：4
变动：更改

事项: 准备演讲稿
日期: 2024-03-15
时间段: 15:00-17:00
类型: 任务事项
截止日期：2024-03-16
重要程度：4
变动：新增

事项: 项目会议
日期: 2024-03-16
时间段: 15:00-16:00
类型: 固定日程
截止日期：2024-03-16
重要程度：5
变动：新增


请根据以上指令完成以下任务规划，并严格遵守输出格式：
【输入】
当前时间：{current_time}

当前时间表：
{schedule}

新增任务：
{prompt}

【输出】
"""

        response = client.chat.completions.create(
            model=model,
            messages=[
                {"role": "system", "content": "您是一名专业的时间规划师，精通GTD工作法和敏捷项目管理。请根据用户提供的待办事项和现有时间表，完成以下任务："},
                {"role": "user", "content": prompt},
            ],
            stream=False,
            max_tokens=512,
            temperature=0.5
        )

        return response.choices[0].message.content
    
    except Exception as e:
        return f"Error querying DeepSeek API: {str(e)}"



In [ ]:
user_prompt = input("你有什么想放进计划表的东西吗？")
response = query_deepseek(user_prompt)
print("\n模型回复：")
print(response)


Response:
新任务：做程序给周姐  
做程序给周姐 → 拆分为：  
- 编写代码框架（预计需1.5小时，时段：2025-03-02 09:00-10:30）  
- 测试与调试（预计需0.5小时，时段：2025-03-02 10:40-11:10）  

日程建议：  
事项: 编写代码框架  
日期: 2025-03-02  
时间段: 09:00-10:30  
类型: 任务事项  
截止日期：2025-03-02  
重要程度：5  
变动：新增  

事项: 测试与调试  
日期: 2025-03-02  
时间段: 10:40-11:10  
类型: 任务事项  
截止日期：2025-03-02  
重要程度：5  
变动：新增  

（现有日程「入职培训」无需调整，保留原时间）  

---

### 逻辑说明：  
1. **任务拆分**：根据「单任务≤120分钟」规则，将编程任务拆分为1.5小时代码框架+0.5小时测试，总耗时符合用户需求  
2. **时段分配**：  
   - 优先占用次日上午专注时段（09:00-12:30），满足「截止时间近者优先」原则  
   - 插入10分钟缓冲（10:30-10:40），符合间隔要求  
3. **应急缓冲**：预留3月2日11:10-12:30作为自由缓冲区（占原任务总时长15%）  
4. **弹性时段保留**：未占用19:00-21:00时段，确保突发需求可处理


任务拆解：
按MECE原则分解为最小可执行单元（每个子任务≤45分钟）
标注必要的前置条件（如：需要网络/专注环境）

优先级矩阵：
艾森豪威尔矩阵（紧急×重要）确定大类优先级
子任务使用MoSCoW法则（Must/Should/Could/Won't）标注
跨任务依赖关系用甘特图逻辑处理